# 5. Baseline Strategy Backtest

EMA(5/15) crossover with regime filter.


In [ ]:

import pandas as pd
from pathlib import Path
from src.backtest import backtest

DATA_DIR = Path("../data")
df = pd.read_csv(DATA_DIR/"nifty_features_5min.csv", parse_dates=["timestamp"])
df = df.dropna(subset=["spot_ret","ema_5","ema_15"]).reset_index(drop=True)

if "regime" not in df.columns:
    df["regime"] = 0

df["cross_up"] = (df["ema_5"] > df["ema_15"]) & (df["ema_5"].shift(1) <= df["ema_15"].shift(1))
df["cross_dn"] = (df["ema_5"] < df["ema_15"]) & (df["ema_5"].shift(1) >= df["ema_15"].shift(1))

df["signal"] = 0
df.loc[df["cross_up"] & (df["regime"] == 1), "signal"] = 1
df.loc[df["cross_dn"] & (df["regime"] == -1), "signal"] = -1

total_return, sharpe = backtest(df)
print("Total Return:", total_return)
print("Sharpe:", sharpe)


In [ ]:

from pathlib import Path
import pandas as pd

RES_DIR = Path("../results")
RES_DIR.mkdir(exist_ok=True)

pd.DataFrame([{"total_return": float(total_return), "sharpe": float(sharpe)}]).to_csv(RES_DIR/"baseline_metrics.csv", index=False)
print("Saved results/baseline_metrics.csv")
